![alt text](./pageheader_rose2_babies.jpg)

# Analysis of Pmax in leak-compensated SIPPV-VG recordings

#### Author: Dr Gusztav Belteki

This Notebook analyses the difference between Pmax and PIP and how it affects tidal volume delivery and alarm activity

### Importing the necessary libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# import seaborn as sns
import os
import sys
import pickle
import scipy as sp
from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
# pd.set_option('mode.chained_assignment', None) 

### Importing custom functions from own module

In [ ]:
from gb_loader import *
from gb_transform import *
from gb_stats import *
from gb_visualizer import *

In [ ]:
print("Python version: {}".format(sys.version))
print("IPython version: {}".format(IPython.__version__))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))

### List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'Pmax'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_draeger'

# Serialised data will be read from this folder of hard drive
DIR_READ_1 = '/Volumes/%s/data_dump/draeger/%s' % (DRIVE, 'VG')

# Directory on external drive to read the clinical data from
DIR_READ_2 = '/Users/guszti/ventilation_draeger'  

# Directory on external drive to read the original data from
DIR_READ_3 = '/Volumes/%s/Draeger/service_evaluation_old' % DRIVE  

# Directory to write results and selected images to 
if not os.path.isdir('%s/%s/%s' % (CWD, 'Analyses', TOPIC)):
    os.makedirs('%s/%s/%s' % (CWD, 'Analyses', TOPIC))
DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', TOPIC)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ_1

In [ ]:
DIR_READ_2

In [ ]:
DIR_READ_3

In [ ]:
DIR_WRITE

In [ ]:
DATA_DUMP

## Import ventilator data

### Import 'slow_measurements' data from pickle archive

In [ ]:
with open('%s/%s.pickle' % (DIR_READ_1, 'slow_measurements_sippv_vg_lc_1'), 'rb') as handle:
    slow_measurements_1 = pickle.load(handle)

with open('%s/%s.pickle' % (DIR_READ_1, 'slow_measurements_sippv_vg_lc_2'), 'rb') as handle:
    slow_measurements_2 = pickle.load(handle)

In [ ]:
slow_measurements = {**slow_measurements_1, **slow_measurements_2}
del slow_measurements_1; del slow_measurements_2

In [ ]:
len(slow_measurements)

In [ ]:
recordings = sorted(slow_measurements.keys())
print(recordings)

### Remove recordings which were < 12 hours duration

*DG017* did not have 12 hours **continuous** SIPPV-VG recording as it had HFOV periods and SIPPV without VG in between. Therefore, we remove it from the recordings.

In [ ]:
del slow_measurements['DG017']
recordings = sorted(slow_measurements.keys())

### Remove the only recording where muscle relaxation was used (DG018_1)

In [ ]:
del slow_measurements['DG018_1']
recordings = sorted(slow_measurements.keys())

In [ ]:
len(recordings)

### Combine DataFrames with selected parameters into one

In [ ]:
columns_to_keep = ['recording', 'VTmand_kg', 'VTset_kg', 'VT_diff', 'PIP',  'Pmax',  'P_diff',
                    'RR', 'RR_set', 'leak%']
total = []
for recording in recordings:
    total.append(slow_measurements[recording][columns_to_keep])
slow_measurements_all = pd.concat(total)    

In [ ]:
print(sorted(slow_measurements_all['recording'].unique()))

In [ ]:
len(sorted(slow_measurements_all['recording'].unique()))

### Import clinical details for these recordings

In [ ]:
clinical_details = pd.read_excel('%s/data_grabber_patient_data_combined_all.xlsx' % DIR_READ_2)
clinical_details.index = clinical_details['Recording']

In [ ]:
# This is needed if we only want to consider a subset of the recordings
clinical_details = clinical_details.reindex(recordings)

In [ ]:
clinical_details['Recording start'] = pd.to_datetime(clinical_details['Recording period'].apply(lambda x: x[:10]),
                dayfirst = True)
clinical_details['Recording end'] = pd.to_datetime(clinical_details['Recording period'].apply(lambda x: x[11:]),
                dayfirst = True)
clinical_details['Postnatal age'] = clinical_details['Recording start'] - clinical_details['Date of birth']
clinical_details['Corrected gestation'] = clinical_details['Gestation'] + \
                clinical_details['Postnatal age'].astype(int) / (1E+9 * 3600 * 24 * 7)
clinical_details['Corrected gestation'] = round(clinical_details['Corrected gestation'], 2)

In [ ]:
clinical_details.info()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_details.xlsx'))
clinical_details.to_excel(writer,'clinical details')
writer.save()

In [ ]:
pars = ['Gestation', 'Corrected gestation', 'Birth weight', 'Current weight']

clinical_details_stats = round(DataFrame([clinical_details[pars].median(),
                                     clinical_details[pars].min(),
                                     clinical_details[pars].max()]).T, 2)
clinical_details_stats.columns = ['median', 'min', 'max']
# clinical_details_stats = np.floor(clinical_details_stats)

In [ ]:
clinical_details_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_details_stats.xlsx'))
clinical_details_stats.to_excel(writer,'clinical details')
writer.save()

In [ ]:
sum(np.floor(clinical_details['Gestation']) <= 27)

In [ ]:
sum(np.floor(clinical_details['Gestation']) <= 32)

In [ ]:
sum(np.floor(clinical_details['Gestation']) <= 36)

### Length of recordings

In [ ]:
recording_duration = {}
for recording in recordings:
    recording_duration[recording] = len(slow_measurements[recording])

recording_duration = DataFrame([recording_duration]).T
recording_duration.columns = ['seconds']
recording_duration;

### Remove missing data

In [ ]:
slow_measurements_all.isnull().sum().sort_values()

In [ ]:
# What proportion of the data missing in the various columns
slow_measurements_all.isnull().sum().sort_values() / len(slow_measurements_all)

In [ ]:
# This represent only a couple percentage of the data and can be removed

a = len(slow_measurements_all)
print('Before removal: %d rows' % a)

slow_measurements_all.dropna(inplace = True)

b = len(slow_measurements_all)
print('After removal: %d rows' % b)
print('Removed %d rows' % (a-b))
print('Removed %.3f percent of the rows' % ((a-b) / a * 100))

In [ ]:
# Now there are no missing data
slow_measurements_all.isnull().sum().sort_values()

### Remove those rows where  VTmand_kg > 20 mL/kg or when RR > 120 mL/kg as these seem to be outliers (probably the circuit was open or sensor problems) rather than reflecting real effective or expiratory tidal volumes

In [ ]:
c = len(slow_measurements_all)
print('Before removal: %d rows' % c)
 
slow_measurements_all = slow_measurements_all[slow_measurements_all['VTmand_kg'] <= 20]
slow_measurements_all = slow_measurements_all[slow_measurements_all['RR'] <= 120]

d = len(slow_measurements_all)
print('After removal: %d rows' % d)
print('Removed %d rows' % (c-d))
print('Removed %.3f percent of the rows' % ((c-d) / a * 100))

### Individual and combined length of the recordings after the cleanup

In [ ]:
recording_duration = {}
for recording in recordings:
    recording_duration[recording] = len(slow_measurements[recording])

recording_duration = DataFrame([recording_duration]).T
recording_duration.columns = ['seconds']
recording_duration

In [ ]:
recording_time_total = recording_duration.sum()[0]

In [ ]:
print('total recoding time = %d seconds' % recording_time_total)
print('total recoding time = %.2f hours' % (recording_time_total / 3600))
print('total recoding time = %.2f days' % (recording_time_total / (3600 * 24)))
print('mean recoding time = %.2f hours' % ((recording_time_total / 3600) / len(recordings)))
print('median recoding time = %.2f hours' % (recording_duration.median() / 3600))

In [ ]:
# Write results to Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'recording_duration_Pmax.xlsx'))
recording_duration.to_excel(writer,'recording_duration')
writer.save()

## Exploratory data analysis on the final combined dataset

In [ ]:
len(slow_measurements_all)

In [ ]:
slow_measurements_all.set_index('recording', append = True, inplace = True)

In [ ]:
slow_measurements_all = slow_measurements_all.swaplevel(1,0)

In [ ]:
slow_measurements_all.head()

In [ ]:
slow_measurements_all.info()

### Range and distribution of Respiratory rate

In [ ]:
bins = list(np.arange(0, 121, 5))

cats_RR = pd.cut(slow_measurements_all['RR'], 
                              bins = bins, right = False)
cats_RR.value_counts().sort_index()

In [ ]:
(cats_RR.value_counts() / len(cats_RR)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_RR.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'RR', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (1/min)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('RR', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'RR_distribution', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_RR.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'RR', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (1/min)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('RR', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'RR_distribution_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Range and distribution of VTmand

In [ ]:
bins = np.arange(0, 21, 1)

cats_VTmand = pd.cut(slow_measurements_all['VTmand_kg'], 
                              bins = bins, right = False)
cats_VTmand.value_counts().sort_index()

In [ ]:
(cats_VTmand.value_counts() / len(cats_VTmand)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_VTmand.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VTmand', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTmand', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTmand_distribution', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_VTmand.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'VTmand', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTmand', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTmand_distribution_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Range and distribution of VTdiff

In [ ]:
bins = np.arange(-6, 17, 1)

cats_VTdiff = pd.cut(slow_measurements_all['VT_diff'], 
                              bins = bins, right = False)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-4, 5, 1))

cats_VTdiff = pd.cut(slow_measurements_all['VT_diff'], 
                              bins = bins, right = False)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_2_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Range and distribution of PIP

In [ ]:
bins = np.arange(0, 51, 2.5)

cats_PIP = pd.cut(slow_measurements_all['PIP'], 
                              bins = bins, right = False)
cats_PIP.value_counts().sort_index()

In [ ]:
(cats_PIP.value_counts() / len(cats_PIP)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_PIP.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'PIP', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mbar)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('PIP', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PIP_distribution', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_PIP.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'PIP', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mbar)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('PIP', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PIP_distribution_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Range and distribution of Pdiff

In [ ]:
bins = list(np.arange(0, 31, 2)) + [40]

cats_P_diff = pd.cut(slow_measurements_all['P_diff'], 
                              bins = bins, right = False)
cats_P_diff.value_counts().sort_index()

In [ ]:
(cats_P_diff.value_counts() / len(cats_P_diff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_P_diff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'Pdiff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mbar)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'P_diff_distribution', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(0, 31, 5))

cats_P_diff = pd.cut(slow_measurements_all['P_diff'], 
                              bins = bins, right = False)
cats_P_diff.value_counts().sort_index()

In [ ]:
(cats_P_diff.value_counts() / len(cats_P_diff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_P_diff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'Pdiff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mbar)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'P_diff_distribution_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Range and distribution of leak%

In [ ]:
bins = list(np.arange(0, 101, 10))

cats_leak = pd.cut(slow_measurements_all['leak%'], 
                              bins = bins, right = False)
cats_leak.value_counts().sort_index()

In [ ]:
(cats_leak.value_counts() / len(cats_leak)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'leak percentage', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (%)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('leak percentage', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_distribution', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Calculate how much the VTdiff is for various bins of Pdiff

In [ ]:
bins = list(np.arange(0, 31, 5))

cats_P_diff = pd.cut(slow_measurements_all['P_diff'], 
                              bins = bins, right = False)
cats_P_diff.value_counts().sort_index()

In [ ]:
slow_measurements_all_binned_P_diff = slow_measurements_all.groupby(cats_P_diff)

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (4,4))
data = []
for name, group in slow_measurements_all_binned_P_diff:
    data.append(group['VT_diff'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.xlabel('Pdiff (mbar)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Pdiff_VTdiff',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Calculate how much the PIP is for various bins of VT_diff

In [ ]:
bins = list(np.arange(-4, 5, 1))

cats_VT_diff = pd.cut(slow_measurements_all['VT_diff'], 
                              bins = bins, right = False)
cats_VT_diff.value_counts().sort_index()

In [ ]:
slow_measurements_all_binned_VT_diff = slow_measurements_all.groupby(cats_VT_diff)

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (6,4))
data = []
for name, group in slow_measurements_all_binned_VT_diff:
    data.append(group['PIP'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.xlabel('VTdiff (mL/kg)', size = 14)
plt.ylabel('PIP (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_PIP',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
VT_diff_stats = slow_measurements_all_binned_VT_diff.describe(percentiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95])
VT_diff_stats

## Aggregate statistics of ventilator parameter in each recording

In [ ]:
grouped = slow_measurements_all.groupby('recording')

In [ ]:
# Calculate all descriptive stats together
all_stats = round(grouped.describe(percentiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]), 2)
all_stats

## Additional statistics

### Recording duration

In [ ]:
recording_duration['hours'] = recording_duration['seconds'] / 3600

In [ ]:
recording_duration.columns = [['Recording duration', 'Recording duration'],
                              ['seconds', 'hours']]

In [ ]:
recording_duration

### Calculate statistics about Pdiff 

In [ ]:
all_stats['P_diff']['50%'].median(), all_stats['P_diff']['50%'].min(), all_stats['P_diff']['50%'].max()

#### P_diff

In [ ]:
slow_measurements_all_not_limited_1 = \
    slow_measurements_all[(slow_measurements_all['PIP'] > (slow_measurements_all['Pmax'] - 10))]
    
print('PIP was <10 mbar below Pmax  in %.2f percent of inflations' % \
(len(slow_measurements_all_not_limited_1) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_not_limited_2 = \
    slow_measurements_all[(slow_measurements_all['PIP'] > (slow_measurements_all['Pmax'] - 5))]
    
print('PIP was <5 mbar below Pmax  in %.2f percent of inflations' % \
(len(slow_measurements_all_not_limited_2) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_3 = \
        slow_measurements_all[(slow_measurements_all['PIP'] >= slow_measurements_all['Pmax'])]
    
print('Pmax was reached or exceeded in %.2f percent of inflations' % \
(len(slow_measurements_all_limited_3) / len(slow_measurements_all) * 100))

##### VT_diff

In [ ]:
slow_measurements_all_limited_4 = \
        slow_measurements_all[(slow_measurements_all['VT_diff'] < 0)]
    
print('The targeted VT was not reached in %.2f percent of inflations' % \
(len(slow_measurements_all_limited_4) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_5 = \
        slow_measurements_all[(slow_measurements_all['VT_diff'] < -1)]
    
print('The targeted VT was not reached by >1 mL/kg in %.2f percent of inflations' % \
(len(slow_measurements_all_limited_5) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_6 = \
        slow_measurements_all[(slow_measurements_all['VT_diff'] < -2)]
    
print('The targeted VT was not reached by >2 mL/kg in %.2f percent of inflations' % \
(len(slow_measurements_all_limited_6) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_7 = \
        slow_measurements_all[(slow_measurements_all['VT_diff'] < -3)]
    
print('The targeted VT was not reached by >3 mL/kg in %.2f percent of inflations' % \
(len(slow_measurements_all_limited_7) / len(slow_measurements_all) * 100))

##### P_diff and VT_diff

In [ ]:
slow_measurements_all_limited_8 = \
        slow_measurements_all[(slow_measurements_all['PIP'] >= slow_measurements_all['Pmax']) & 
                              (slow_measurements_all['VT_diff'] < 0) ]
    
print('Pmax was reached or exceeded AND set VT was not achieved  in %.2f percent of inflations' % \
(len(slow_measurements_all_limited_8) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_9 = \
        slow_measurements_all[(slow_measurements_all['PIP'] >= slow_measurements_all['Pmax']) & 
                              (slow_measurements_all['VT_diff'] <= -1) ]
    
print('''Pmax was reached or exceeded AND set VT was not achieved by at least 1 mL/kg 
 in %.2f percent of inflations''' % (len(slow_measurements_all_limited_9) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_10 = \
        slow_measurements_all[(slow_measurements_all['PIP'] >= slow_measurements_all['Pmax']) & 
                              (slow_measurements_all['VT_diff'] <= -2) ]
    
print('''Pmax was reached or exceeded AND set VT was not achieved by at least 2 mL/kg 
 in %.2f percent of inflations''' % (len(slow_measurements_all_limited_10) / len(slow_measurements_all) * 100))

In [ ]:
slow_measurements_all_limited_11 = \
        slow_measurements_all[(slow_measurements_all['PIP'] >= slow_measurements_all['Pmax']) & 
                              (slow_measurements_all['VT_diff'] <= -3) ]
    
print('''Pmax was reached or exceeded AND set VT was not achieved by at least 3 mL/kg 
 in %.2f percent of inflations''' % (len(slow_measurements_all_limited_11) / len(slow_measurements_all) * 100))

##### Percentage of inflations with Pmax reached which do not achieve target VT by a partuculat value

In [ ]:
print('''VT was not achieved in %.2f percent of inflations when Pmax was reached''' 
      % (len(slow_measurements_all_limited_8) / len(slow_measurements_all_limited_3) * 100))

In [ ]:
print('''VT was not achieved by at least 1 mL/kg in %.2f percent of inflations when Pmax was reached''' 
      % (len(slow_measurements_all_limited_9) / len(slow_measurements_all_limited_3) * 100))

In [ ]:
print('''VT was not achieved by at least 2 mL/kg in %.2f percent of inflations when Pmax was reached''' 
      % (len(slow_measurements_all_limited_10) / len(slow_measurements_all_limited_3) * 100))

In [ ]:
print('''VT was not achieved by at least 3 mL/kg in %.2f percent of inflations when Pmax was reached''' 
      % (len(slow_measurements_all_limited_11) / len(slow_measurements_all_limited_3) * 100))

#### Create a Dataframe for category 9 (Pmax was reached and VT was not achieved by >1 mL/kg)

In [ ]:
def filter(dframe):
    return  dframe[(dframe['PIP'] >= dframe['Pmax']) & (dframe['VT_diff'] < -1)]

In [ ]:
limited_inflations = grouped.apply(filter)

In [ ]:
len(limited_inflations)

In [ ]:
limited_inflations.head();

In [ ]:
limited_inflations_stats = DataFrame(limited_inflations.groupby('recording').size())
limited_inflations_stats['%']= \
    round((limited_inflations.groupby('recording').size() / grouped.size() * 100), 2)

limited_inflations_stats.columns = [['limited inflations', 'limited inflations'], 
                                    ['count', '%']] 
limited_inflations_stats

### Add how many times the Pmax was changed during the recordings

In [ ]:
# Import ventilator settings

vent_settings = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ_3, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = slow_setting_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ_3, recording, filename) for filename in files]
    vent_settings[recording] =  data_loader(fnames)

In [ ]:
# Limit ventilation settings to the duration of the recordings

for recording in recordings:
        
    if slow_measurements[recording].index[0] <= vent_settings[recording].index[0]:
        continue
    else:
        vent_settings[recording] = vent_settings[recording][slow_measurements[recording].index[0] : ]
        
    if slow_measurements[recording].index[-1] >= vent_settings[recording].index[-1]:
        continue
    else:
        vent_settings[recording] = vent_settings[recording][ : slow_measurements[recording].index[-1]]

In [ ]:
columns_to_keep  = ['Id', 'Value Old', 'Value New']

total = []
for recording in recordings:
    to_keep = vent_settings[recording][columns_to_keep].copy()
    to_keep['recording'] = recording
    total.append(to_keep)
vent_settings_all = pd.concat(total)    

In [ ]:
vent_setting_grouped = vent_settings_all.groupby('recording')

In [ ]:
# Create a Dataframe for category 2

def filter_2(dframe):
    return  dframe[dframe['Id'] == 'Pmax']

In [ ]:
Pmax_changes = vent_setting_grouped.apply(filter_2)
Pmax_changes.drop(['recording'], axis = 1, inplace  = True)

In [ ]:
# The first entry in the vent_settings DataFrames are the initial settings not changes made 

Pmax_changes_no_initial = {}

for recording in Pmax_changes.index.levels[0]:
    # print(recording)
    a = Pmax_changes.loc[recording]
    try:
        Pmax_changes_no_initial[recording] = a.drop(a.index[0], inplace=False)
    except:
        continue # some of the recordings have no change in vent settings at all
        
Pmax_changes_no_initial = pd.concat(Pmax_changes_no_initial)
# some tables are compiled from more than one file, for these there will be a 
# second 'initial value' marked by an Old Value of 'nan'; remove this
Pmax_changes_no_initial.dropna(inplace = True) 
Pmax_changes_no_initial.index.levels[0].name = 'recording'

In [ ]:
len(Pmax_changes_no_initial)

In [ ]:
recording_duration

In [ ]:
Pmax_changes_stats = DataFrame(Pmax_changes_no_initial.groupby('recording').size(), 
            columns = ['Pmax changes (count)'])
Pmax_changes_stats['Pmax changes (count[day])'] = \
    round(Pmax_changes_stats['Pmax changes (count)'] / 
          (recording_duration['Recording duration']['seconds'] / (3660 * 24)), 2)
Pmax_changes_stats.columns = [['Pmax changes', 'Pmax changes'],
                             ['count', 'count/day' ]]
Pmax_changes_stats

In [ ]:
Pmax_changes_stats.median()

### Number of low tidal volume alarms

In [ ]:
alarm_states = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ_3, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = alarm_state_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ_3, recording, filename) for filename in files]
    alarm_states[recording] =  data_loader(fnames)

In [ ]:
# Limit alarm events for the duration of the recordings

for recording in recordings:
    
    if slow_measurements[recording].index[0] <= alarm_states[recording].index[0]:
        continue
    else:
        alarm_states[recording] = alarm_states[recording][slow_measurements[recording].index[0] : ]
        
    if slow_measurements[recording].index[-1] >= alarm_states[recording].index[-1]:
        continue
    else:
        alarm_states[recording] = alarm_states[recording][ : slow_measurements[recording].index[-1]]

In [ ]:
alarm_count = {}
alarm_count_total = 0

for recording in recordings:
    a = alarm_states[recording]
    b = a[a['Name'].isin(['Tidal volume < low Limit', 'Tube obstructed','Volume not constant'])]
    c = b[b['State New'] == 'Active']
    alarm_count[recording] = len(c)
    
alarm_count = DataFrame([alarm_count]).T
alarm_count.index.name = 'recording'
alarm_count.columns = ['alarm count']

alarm_count['alarm count/hour'] = round(alarm_count['alarm count'] /\
                    (recording_duration['Recording duration']['seconds'] / 3660), 2)
alarm_count.columns = [['Low VT alarm', 'Low VT alarm'],
                       ['count', 'count/hour']]
alarm_count

### Combine additional statistics

In [ ]:
additional_stats = pd.concat([recording_duration, limited_inflations_stats,
                              Pmax_changes_stats, alarm_count], axis = 1)
additional_stats.replace(np.nan, 0, inplace = True)

In [ ]:
additional_stats

In [ ]:
combined_stats = pd.concat([all_stats, additional_stats], axis = 1)

In [ ]:
combined_stats

In [ ]:
sum(combined_stats['Pmax']['max'] > 20)

In [ ]:
sum(combined_stats['Pmax']['max'] > 30)

In [ ]:
sum(combined_stats['Pmax']['max'] > 40)

In [ ]:
# Write statistics into  Excel file

writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all.xlsx'))
all_stats.to_excel(writer, 'all_stats')
additional_stats.to_excel(writer, 'additional_stats')
combined_stats.to_excel(writer, 'combined_stats')
writer.save()

### EDA on this DataFrame

In [ ]:
combined_stats.describe()

In [ ]:
correlation_all = round(combined_stats.corr(method = 'spearman'), 2)
correlation_all;

In [ ]:
# Write statistics into  Excel file

writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'correlation_all.xlsx'))
correlation_all.to_excel(writer, 'stats')
writer.save()

In [ ]:
combined_stats.corr(method='spearman')['P_diff']['50%'].sort_values(ascending = False)

In [ ]:
from scipy.stats import pearsonr

def corr_pearson(x, y):

    '''
    input: two numeric arrays of the same size

    returns: a tuple of 
    1. Pearson's correlation coefficient: r
    2. low and high 95% confidence intervals or r (two values)
    3. Coefficient of determination: r^2
    4. p-value of correlation

    '''
    
    assert len(x) == len(y)
    
    r, p = pearsonr(x, y)
    f = 0.5*np.log((1+r)/(1-r))
    se = 1/np.sqrt(len(x)-3)
    ucl = f + 1.96 * se
    lcl = f - 1.96 * se

    lcl = (np.exp(2*lcl) - 1) / (np.exp(2*lcl) + 1)
    ucl = (np.exp(2*ucl) - 1) / (np.exp(2*ucl) + 1)

    return r , lcl, ucl , r*r, p

In [ ]:
from scipy.stats import spearmanr

def corr_spearman(x, y):

    '''
    input: two numeric arrays of the same size

    returns: a tuple of 
    1. Spearman's correlation coefficient: r
    2. low and high 95% confidence intervals or r (two values)
    3. Coefficient of determination: r^2
    4. p-value of correlation

    '''
    
    assert len(x) == len(y)
    
    r, p = spearmanr(x, y)
    f = 0.5*np.log((1+r)/(1-r))
    se = 1/np.sqrt(len(x)-3)
    ucl = f + 1.96 * se
    lcl = f - 1.96 * se

    lcl = (np.exp(2*lcl) - 1) / (np.exp(2*lcl) + 1)
    ucl = (np.exp(2*ucl) - 1) / (np.exp(2*ucl) + 1)

    return r , lcl, ucl , r*r, p

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(combined_stats['limited inflations']['%'], 
                                      combined_stats['Low VT alarm']['count/hour'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.3f}'.format(r , lcl, ucl , r2, p))

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(combined_stats['limited inflations']['%'], 
                                      combined_stats['P_diff']['50%'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.3f}'.format(r , lcl, ucl , r2, p))

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(combined_stats['Low VT alarm']['count/hour'], 
                                      combined_stats['P_diff']['50%'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.3f}'.format(r , lcl, ucl , r2, p))

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(combined_stats['VT_diff']['50%'], 
                                      combined_stats['P_diff']['50%'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.3f}'.format(r , lcl, ucl , r2, p))

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(combined_stats['Pmax']['50%'], 
                                      combined_stats['P_diff']['50%'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.3f}'.format(r , lcl, ucl , r2, p))

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(combined_stats['Pmax']['50%'], 
                                      combined_stats['PIP']['50%'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.3f}'.format(r , lcl, ucl , r2, p))

### Resample data to get the 1-minute medians and 1-hourly medians

In [ ]:
def transform_1(dframe):
    a = dframe.reset_index(0)
    return a.resample('1min').median()

In [ ]:
def transform_2(dframe):
    a = dframe.reset_index(0)
    return a.resample('1H').median()

In [ ]:
%%time

slow_measurements_all_1min_median = grouped.apply(transform_1)

In [ ]:
slow_measurements_all_1min_median.head()

In [ ]:
%%time

slow_measurements_all_1hour_median = grouped.apply(transform_2)

In [ ]:
slow_measurements_all_1hour_median.head()

In [ ]:
print(recordings)

In [ ]:
slow_measurements_all.head()

In [ ]:
slow_measurements_all_1min_median.head()

In [ ]:
slow_measurements_all_1hour_median.head()

### Export PIP and VTmand graphs as images

In [ ]:
# PIP and Pmax, 1/sec data

for recording in recordings:

    print('Saving images for %s' % recording)
    rec = recording
    filetype = 'jpg'
    dpi = 300
    
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements[rec]['PIP'].plot(ax = ax, color = 'red')
    slow_measurements[rec]['Pmax'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

    import matplotlib.dates as mdates
    myFmt = mdates.DateFormatter('%H:%M')
    #ax.xaxis.set_major_formatter(myFmt)
    plt.xticks(rotation=0)

    ax.set_ylim(0, slow_measurements[rec]['Pmax'].max() + 10)
    ax.set_xlabel('time', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('%s' % rec,  size = 12, color = 'black')
    ax.legend(['PIP', 'Pmax'], fontsize = 12)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax.tick_params(which = 'both', labelsize=12)
    plt.tight_layout()
    plt.close()

    fig.savefig('%s/%s_%s_%s.%s' % (DATA_DUMP, rec,  'PIP_Pmax', '1sec', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);
                                                       

In [ ]:
# VTmand and VTset, 1/sec data

for recording in recordings:

    print('Saving images for %s' % recording)
    rec = recording
    filetype = 'pdf'
    dpi = 200
    
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements[rec]['VTmand_kg'].plot(ax = ax, color = 'blue')
    slow_measurements[rec]['VTset_kg'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

    import matplotlib.dates as mdates
    myFmt = mdates.DateFormatter('%H:%M')
    #ax.xaxis.set_major_formatter(myFmt)
    plt.xticks(rotation=0)

    ax.set_ylim(0, slow_measurements[rec]['VTmand_kg'].max() + 10)
    ax.set_xlabel('time', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('%s' % rec,  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTset'], fontsize = 12)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax.tick_params(which = 'both', labelsize=12)
    plt.tight_layout()
    plt.close()

    fig.savefig('%s/%s_%s_%s.%s' % (DATA_DUMP, rec,  'VTmand_VTset', '1sec', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);
                                                       

In [ ]:
# PIP and Pmax, 1-minute medians

for recording in recordings:

    print('Saving images for %s' % recording)
    rec = recording
    filetype = 'pdf'
    dpi = 200

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_all_1min_median.loc[rec]['PIP'].plot(ax = ax, color = 'red')
    slow_measurements_all_1min_median.loc[rec]['Pmax'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')
    plt.xticks(rotation=0)

    ax.set_ylim(0, slow_measurements_all_1min_median.loc[rec]['Pmax'].max() + 10)
    ax.set_xlabel('time', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('%s' % rec,  size = 12, color = 'black')
    ax.legend(['PIP', 'Pmax'], fontsize = 12)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax.tick_params(which = 'both', labelsize=12)
    plt.tight_layout()
    plt.close()

    fig.savefig('%s/%s_%s_%s.%s' % (DATA_DUMP, rec,  'PIP_Pmax', '1min_med', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);
                                                       

In [ ]:
# VTmand and VTset, 1-minute medians

for recording in recordings:

    print('Saving images for %s' % recording)
    rec = recording
    filetype = 'jpg'
    dpi = 400

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_all_1min_median.loc[rec]['VTmand_kg'].plot(ax = ax, color = 'blue')
    slow_measurements_all_1min_median.loc[rec]['VTset_kg'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')
    plt.xticks(rotation=0)

    ax.set_ylim(0, slow_measurements_all_1min_median.loc[rec]['VTmand_kg'].max() + 8)
    ax.set_xlabel('time', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('%s' % rec,  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTset'], fontsize = 12)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax.tick_params(which = 'both', labelsize=12)
    plt.tight_layout()
    plt.close()

    fig.savefig('%s/%s_%s_%s.%s' % (DATA_DUMP, rec,  'VTmand_VTset', '1min_med', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);
                                                       

In [ ]:
# PIP and Pmax, 1-hour medians

for recording in recordings:

    print('Saving images for %s' % recording)
    rec = recording
    filetype = 'pdf'
    dpi = 200

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_all_1hour_median.loc[rec]['PIP'].plot(ax = ax, color = 'red')
    slow_measurements_all_1hour_median.loc[rec]['Pmax'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

    plt.xticks(rotation=0)
    ax.set_ylim(0, slow_measurements_all_1hour_median.loc[rec]['Pmax'].max() + 10)
    ax.set_xlabel('time', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('%s' % rec,  size = 12, color = 'black')
    ax.legend(['PIP', 'Pmax'], fontsize = 12)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax.tick_params(which = 'both', labelsize=12)
    plt.tight_layout()
    plt.close()

    fig.savefig('%s/%s_%s_%s.%s' % (DATA_DUMP, rec,  'PIP_Pmax', '1hour_med', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);
                                                       

In [ ]:
# VTmand and VTset, 1-hour medians

for recording in recordings:

    print('Saving images for %s' % recording)
    rec = recording
    filetype = 'pdf'
    dpi = 200

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_all_1hour_median.loc[rec]['VTmand_kg'].plot(ax = ax, color = 'blue')
    slow_measurements_all_1hour_median.loc[rec]['VTset_kg'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

    plt.xticks(rotation=0)
    ax.set_ylim(0, slow_measurements_all_1hour_median.loc[rec]['VTmand_kg'].max() + 10)
    ax.set_xlabel('time', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('%s' % rec,  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTset'], fontsize = 12)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax.tick_params(which = 'both', labelsize=12)
    plt.tight_layout()
    plt.close()

    fig.savefig('%s/%s_%s_%s.%s' % (DATA_DUMP, rec,  'VTmand_VTset', '1hour_med', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);
                                                       

### Calculate Pmax for every hour as a the mode of the previous hour's PIP + 5 mbar

In [ ]:
# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrames
# purpose - splits the DataFrame into smaller of max size chunkSize (last is smaller)
def splitDataFrameIntoSmaller(df, chunkSize): 
    listOfDf = []
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize].copy())
    return listOfDf

In [ ]:
def Pmax_emulator(frame, window = 3600, overPIP = 5):
    aa = splitDataFrameIntoSmaller(frame[['PIP', 'Pmax', 'P_diff']], window)
    aa[0]['Pmax_calc'] = aa[0]['Pmax'] # for the first hour keep the original Pmax
    for i in range(len(aa)-1): # the last hour's mode does not need to be calculated
        aa[i+1]['Pmax_calc'] = (aa[i]['PIP'].value_counts().argmax() + overPIP)
    
    return aa

In [ ]:
def Pdiff_calculator(dframes):
    
    '''
    dframes = collection of DataFrames
    
    '''
    frame_list = list(dframes)
    Pdiff_list = []
    for frame in frame_list:
        Pdiff = list(frame['Pmax_calc'] - frame['PIP'])
        Pdiff_list.extend(Pdiff)
    return Pdiff_list    

In [ ]:
Pmax_hit_frame = pd.read_excel('%s/Pmax_hit_frame.xlsx' % DIR_WRITE)

In [ ]:
Pmax_hit_frame.index.name = 'Pmax set over PIP (mbar)'

In [ ]:
Pmax_hit_frame.columns.name = 'Frequency of Pmax changes in hours'

In [ ]:
Pmax_hit_frame

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots()
plt.plot(Pmax_hit_frame.loc[5], marker = 'o', label = '5 mbar over PIP'); 
plt.plot(Pmax_hit_frame.loc[10], marker = 'o', label = '10 mbar over PIP'); 
plt.plot(Pmax_hit_frame.loc[15], marker = 'o', label = '15 mbar over PIP'); 

plt.ylim(0,17)
plt.legend()

plt.xlabel('Frequency of adjusting Pmax (hours)', size = 14)
plt.ylabel('%', size = 14)
plt.title('Percentage of inflations when Pmax was reached', size = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Pmax_hit_1',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots()
plt.pcolor(Pmax_hit_frame, cmap =plt.get_cmap('Reds'))
plt.xticks(np.arange(0.5, 12.5), Pmax_hit_frame.columns)
plt.yticks(np.arange(0.5, 11.5), Pmax_hit_frame.index)
plt.ylim(0, 11)
plt.xlabel('Frequency of adjusting Pmax (hours)')
plt.ylabel('Setting Pmax over PIP (mbar)')
plt.title('Percentage of inflations when Pmax was reached')
cb = plt.colorbar()
cb.set_label('% of inflations')

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Pmax_hit_2',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

## Tables and Figures for the paper

### Tables

#### Table 1

In [ ]:
clin_details_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Table_1.xlsx'))
clinical_details_stats.to_excel(writer,'Table_1')
writer.save()

In [ ]:
sum(np.floor(clinical_details['Gestation']) <= 27)

In [ ]:
sum(np.floor(clinical_details['Gestation']) <= 32)

In [ ]:
sum(np.floor(clinical_details['Gestation']) <= 36)

______

#### Table 2 / Supplementary Table 1

In [ ]:
# Write statistics into  Excel file

writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Table_2_Supplementary_Table_1.xlsx'))
combined_stats.to_excel(writer, 'stats')
writer.save()

#### Table 3

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Table_3.xlsx'))
Pmax_hit_frame.to_excel(writer, 'Pmax_over_PIP')
writer.save()

### Figures

#### Figure 1

In [ ]:
rec = 'DG026'
filetype = 'jpg'
dpi = 200

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements[rec]['PIP'].plot(ax = ax, color = 'red')
slow_measurements[rec]['Pmax'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M')
plt.xticks(rotation=0)

ax.set_ylim(0, slow_measurements[rec]['Pmax'].max() + 15)
ax.set_xlabel('time', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('%s' % rec,  size = 12, color = 'black')
ax.legend(['PIP', 'Pmax'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1A', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);                                                                                                          

In [ ]:
rec = 'DG026'
filetype = 'jpg'
dpi = 200

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements[rec]['VTmand_kg'].plot(ax = ax, color = 'blue')
slow_measurements[rec]['VTset_kg'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M')
plt.xticks(rotation=0)

ax.set_ylim(0, slow_measurements[rec]['VTmand_kg'].max() + 10)
ax.set_xlabel('time', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('%s' % rec,  size = 12, color = 'black')
ax.legend(['VTmand', 'VTset'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1B', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);                                                      

In [ ]:
rec = 'DG026'
filetype = 'jpg'
dpi = 200

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements_all_1min_median.loc[rec]['PIP'].plot(ax = ax, color = 'red')
slow_measurements_all_1min_median.loc[rec]['Pmax'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

ax.arrow(pd.to_datetime('2016-01-27 21:15:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-27 22:25:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 04:40:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 12:25:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 17:30:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 18:40:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 23:15:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-29 04:00:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)


plt.xticks(rotation=0)

ax.set_ylim(0, slow_measurements_all_1min_median.loc[rec]['Pmax'].max() + 25)
ax.set_xlabel('time', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('%s' % rec,  size = 12, color = 'black')
ax.legend(['PIP', 'Pmax'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1C', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);                                                      

In [ ]:
rec = 'DG026'
filetype = 'jpg'
dpi = 200

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements_all_1min_median.loc[rec]['VTmand_kg'].plot(ax = ax, color = 'blue')
slow_measurements_all_1min_median.loc[rec]['VTset_kg'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')


ax.arrow(pd.to_datetime('2016-01-27 21:15:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-27 22:25:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 04:40:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 12:25:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 17:30:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 18:40:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-28 23:15:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax.arrow(pd.to_datetime('2016-01-29 04:00:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)


import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M')
plt.xticks(rotation=0)

ax.set_ylim(0, slow_measurements_all_1min_median.loc[rec]['VTmand_kg'].max() + 5)
ax.set_xlabel('time', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('%s' % rec,  size = 12, color = 'black')
ax.legend(['VTmand', 'VTset'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1D', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);                                                      

In [ ]:
rec = 'DG026'
filetype = 'jpg'
dpi = 200

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements_all_1hour_median.loc[rec]['PIP'].plot(ax = ax, color = 'red')
slow_measurements_all_1hour_median.loc[rec]['Pmax'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

plt.xticks(rotation=0)
ax.set_ylim(0, slow_measurements_all_1hour_median.loc[rec]['Pmax'].max() + 10)
ax.set_xlabel('time', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('%s' % rec,  size = 12, color = 'black')
ax.legend(['PIP', 'Pmax'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1E', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);                                                      

In [ ]:
rec = 'DG026'
filetype = 'jpg'
dpi = 200

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements_all_1hour_median.loc[rec]['VTmand_kg'].plot(ax = ax, color = 'blue')
slow_measurements_all_1hour_median.loc[rec]['VTset_kg'].plot(ax = ax, color = 'black', 
                                linewidth = 2, linestyle = '--')

import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M')
plt.xticks(rotation=0)

ax.set_ylim(0, slow_measurements_all_1hour_median.loc[rec]['VTmand_kg'].max() + 5)
ax.set_xlabel('time', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('%s' % rec,  size = 12, color = 'black')
ax.legend(['VTmand', 'VTset'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1F', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

#### Figure 2

In [ ]:
labels = ['0-5', '5-10', '10-15', '10-15', '15-20', '20-25', '25-30']
fig, ax = plt.subplots(figsize = [5,4])
cats_P_diff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'Pdiff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mbar)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = 'vertical')
plt.title('Pdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (5,4))
data = []
for name, group in slow_measurements_all_binned_P_diff:
    data.append(group['VT_diff'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.xlabel('Pdiff (mbar)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_2B',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
fig, ax = plt.subplots(figsize = [8,6])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2C', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (6,4))
data = []
for name, group in slow_measurements_all_binned_VT_diff:
    data.append(group['PIP'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.xlabel('VTdiff (mL/kg)', size = 14)
plt.ylabel('PIP (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_2D',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Figure 3

In [ ]:
dpi = 1200
filetype = 'jpg'

fig, ax = plt.subplots()
plt.plot(Pmax_hit_frame.loc[5], marker = 'o', label = '5 mbar over PIP'); 
plt.plot(Pmax_hit_frame.loc[10], marker = 'o', label = '10 mbar over PIP'); 
plt.plot(Pmax_hit_frame.loc[15], marker = 'o', label = '15 mbar over PIP'); 

plt.ylim(0,17)
plt.legend()
plt.xticks(Pmax_hit_frame.columns)
plt.xlabel('Frequency of adjusting Pmax (hours)', size = 14)
plt.ylabel('%', size = 14)
plt.title('Percentage of inflations when Pmax was reached', size = 16)
plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_3',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Combined Figures for paper

### Figure 1

In [ ]:
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M')
dpi = 1200
filetype = 'jpg'
fig, ax = plt.subplots(3, 2, figsize = (10, 12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.2, wspace=0.3)

rec = 'DG026'

# Figure 1A
slow_measurements[rec]['PIP'].plot(ax = ax[0, 0], color = 'red')
slow_measurements[rec]['Pmax'].plot(ax = ax[0, 0], color = 'black', linewidth = 2, linestyle = '--')
plt.xticks(rotation=0)
ax[0, 0].set_ylim(0, slow_measurements[rec]['Pmax'].max() + 15)
ax[0, 0].set_xlabel('', size = 12, color = 'black')
ax[0, 0].set_ylabel('mbar', size = 12, color = 'black')
ax[0, 0].legend(['PIP', 'Pmax'], fontsize = 12)
ax[0, 0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[0, 0].tick_params(which = 'both', labelsize=12)

# Figure 1B
slow_measurements[rec]['VTmand_kg'].plot(ax = ax[0, 1], color = 'blue')
slow_measurements[rec]['VTset_kg'].plot(ax = ax[0, 1], color = 'black', linewidth = 2, linestyle = '--')
plt.xticks(rotation=0)
ax[0, 1].set_ylim(0, slow_measurements[rec]['VTmand_kg'].max() + 10)
ax[0, 1].set_xlabel('', size = 12, color = 'black')
ax[0, 1].set_ylabel('mL/kg', size = 12, color = 'black')
ax[0, 1].legend(['VTmand', 'VTset'], fontsize = 12)
ax[0, 1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[0, 1].tick_params(which = 'both', labelsize=12)

# Figure 1C
slow_measurements_all_1min_median.loc[rec]['PIP'].plot(ax = ax[1,0], color = 'red')
slow_measurements_all_1min_median.loc[rec]['Pmax'].plot(ax = ax[1,0], color = 'black', 
                                linewidth = 2, linestyle = '--')
ax[1,0].arrow(pd.to_datetime('2016-01-27 21:15:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-27 22:25:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-28 04:40:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-28 12:25:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-28 17:30:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-28 18:40:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-28 23:15:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
ax[1,0].arrow(pd.to_datetime('2016-01-29 04:00:00'), 35, 0, -3, head_width=20, head_length=1, 
         color='black', linestyle='-', lw=2)
plt.xticks(rotation=0)
ax[1,0].set_ylim(0, slow_measurements_all_1min_median.loc[rec]['Pmax'].max() + 25)
ax[1,0].set_xlabel('', size = 12, color = 'black')
ax[1,0].set_ylabel('mbar', size = 12, color = 'black')
ax[1,0].legend(['PIP', 'Pmax'], fontsize = 12)
ax[1,0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[1,0].tick_params(which = 'both', labelsize=12)


# Figure 1D
slow_measurements_all_1min_median.loc[rec]['VTmand_kg'].plot(ax = ax[1,1], color = 'blue')
slow_measurements_all_1min_median.loc[rec]['VTset_kg'].plot(ax = ax[1,1], color = 'black', 
                                linewidth = 2, linestyle = '--')
ax[1,1].arrow(pd.to_datetime('2016-01-27 21:15:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-27 22:25:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-28 04:40:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-28 12:25:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-28 17:30:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-28 18:40:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-28 23:15:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
ax[1,1].arrow(pd.to_datetime('2016-01-29 04:00:00'), 8, 0, -1, head_width=20, head_length=0.3, 
         color='black', linestyle='-', lw=2)
plt.xticks(rotation=0)
ax[1,1].set_ylim(0, slow_measurements_all_1min_median.loc[rec]['VTmand_kg'].max() + 5)
ax[1,1].set_xlabel('', size = 12, color = 'black')
ax[1,1].set_ylabel('mL/kg', size = 12, color = 'black')
ax[1,1].legend(['VTmand', 'VTset'], fontsize = 12)
ax[1,1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[1,1].tick_params(which = 'both', labelsize=12)

# Figure 1E
slow_measurements_all_1hour_median.loc[rec]['PIP'].plot(ax = ax[2,0], color = 'red')
slow_measurements_all_1hour_median.loc[rec]['Pmax'].plot(ax = ax[2,0], color = 'black', 
                                linewidth = 2, linestyle = '--')
plt.xticks(rotation=0)
ax[2,0].set_ylim(0, slow_measurements_all_1hour_median.loc[rec]['Pmax'].max() + 10)
ax[2,0].set_xlabel('time (hours)', size = 12, color = 'black')
ax[2,0].set_ylabel('mbar', size = 12, color = 'black')
ax[2,0].legend(['PIP', 'Pmax'], fontsize = 12)
ax[2,0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[2,0].tick_params(which = 'both', labelsize=12)

# Figure 1F
slow_measurements_all_1hour_median.loc[rec]['VTmand_kg'].plot(ax = ax[2,1], color = 'blue')
slow_measurements_all_1hour_median.loc[rec]['VTset_kg'].plot(ax = ax[2,1], color = 'black', 
                                linewidth = 2, linestyle = '--')
plt.xticks(rotation=0)
ax[2,1].set_ylim(0, slow_measurements_all_1hour_median.loc[rec]['VTmand_kg'].max() + 5)
ax[2,1].set_xlabel('time (hours)', size = 12, color = 'black')
ax[2,1].set_ylabel('mL/kg', size = 12, color = 'black')
ax[2,1].legend(['VTmand', 'VTset'], fontsize = 12)
ax[2,1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[2,1].tick_params(which = 'both', labelsize=12)

fig.text(0.01, 0.98, 'A', fontsize = 16); fig.text(0.01, 0.66, 'C', fontsize = 16)
fig.text(0.01, 0.33, 'E', fontsize = 16)
fig.text(0.51, 0.98, 'B', fontsize = 16); fig.text(0.51, 0.66, 'D', fontsize = 16)
fig.text(0.51, 0.33, 'F', fontsize = 16)

plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);                                                                                                          

### Figure 2

In [ ]:
dpi = 1200
filetype = 'jpg'

Pdiff_labels = ['0-5', '5-10', '10-15', '10-15', '15-20', '20-25', '25-30']
VTdiff_labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(2, 2, figsize = (10, 8))
fig.subplots_adjust(left=0.2, bottom=0.2, right=None, top=None, hspace=0.2, wspace=0.5)

# Figure 2A
cats_P_diff.value_counts().sort_index().plot(ax = ax[0,0], kind = 'bar', logy = False, 
                        title = 'Pdiff', color = 'black', alpha = 0.6, fontsize = 14)
ax[0,0].set_ylabel('number of inflations', size = 14)
ax[0,0].set_xlabel('', size = 14)
ax[0,0].set_xticklabels('', rotation = '90', size = 14)
ax[0,0].set_ylim([0, 2500000])
ax[0,0].set_title('', fontsize = 16)
ax[0,0].grid('on')

# Figure 2B
data = []
for name, group in slow_measurements_all_binned_P_diff:
    data.append(group['VT_diff'].values)
ax[1,0].boxplot(data, whis = [5, 95], showfliers = False, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[1,0].set_xticks(np.arange(1, len(data)+1))
ax[1,0].set_xticklabels(Pdiff_labels, rotation = '90', size = 14)
ax[1,0].set_xlabel('Pdiff (mbar)', fontsize = 14)
ax[1,0].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[1,0].grid('on')

# Figure 2C
cats_VTdiff.value_counts().sort_index().plot(ax = ax[0,1], kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.6, fontsize = 14)
ax[0,1].set_xticklabels('', rotation = '90', size = 14)
ax[0,1].set_xlabel('', size = 14)
ax[0,1].set_ylabel('', size = 14)
ax[0,1].set_ylim([0, 2500000])
ax[0,1].set_title('', fontsize = 16)
ax[0,1].grid('on')

# Figure 2D
data = []
for name, group in slow_measurements_all_binned_VT_diff:
    data.append(group['PIP'].values)
ax[1,1].boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[1,1].set_xticks(np.arange(1, len(data)+1))
ax[1,1].set_xticklabels(VTdiff_labels, rotation = '90', size = 14)
ax[1,1].set_xlabel('VTdiff (mL/kg)', size = 14)
ax[1,1].set_ylabel('PIP (mbar)', size = 14)
ax[1,1].grid('on')

fig.text(0.05, 0.96, 'A', fontsize = 16); fig.text(0.51, 0.96, 'C', fontsize = 16)
fig.text(0.05, 0.53, 'B', fontsize = 16); fig.text(0.51, 0.53, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);